# Tabla 3 Zotenko

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pylab
import math
import pandas as pd
from io import StringIO
import random
import scipy
import operator

In [2]:
G_APMS = nx.read_edgelist('yeast_AP-MS.txt')
G_Y2H = nx.read_edgelist('yeast_Y2H.txt')
G_LIT = nx.read_edgelist('yeast_LIT.txt')

G_LIT_Reguly = nx.Graph()
data_lit_reguly = np.genfromtxt("yeast_LIT_Reguly.txt",dtype='unicode',delimiter = '\t')
yeast_LIT_Reguly_raw = [data_lit_reguly[1:,0],data_lit_reguly[1:,1]]

i=1
while i<len(data_lit_reguly[1:,0]):
    G_LIT_Reguly.add_edge(data_lit_reguly[i,0],data_lit_reguly[i,1])
    i+=1

In [3]:
G_ESSENTIAL = nx.Graph()
data_G_ESSENTIAL = np.genfromtxt("Essential_ORFs_paperHe.txt",dtype='unicode',delimiter = '\t')
yeast_G_ESSENTIAL_raw = [data_G_ESSENTIAL[2:,1]]

i = 0
l = len(yeast_G_ESSENTIAL_raw[0][:])

while i < l:
    proteina = yeast_G_ESSENTIAL_raw[0][i]
    yeast_G_ESSENTIAL_raw[0][i] = proteina.rstrip(" ")
    i+=1

In [4]:
def agrega_esencialidad(red,lista_esencial):
    
    j = 0
    l = len(red)
    while j < l:
        nombre = list(red)[j]
        red.node[nombre]['esencialidad']=0
        j += 1
    
    for i in lista_esencial[0]:
        if (i in red):
            red.node[i]['esencialidad']=1
    return

Le agregamos esencialidad a los nodos

In [5]:
agrega_esencialidad(G_APMS,yeast_G_ESSENTIAL_raw)
agrega_esencialidad(G_Y2H,yeast_G_ESSENTIAL_raw)
agrega_esencialidad(G_LIT,yeast_G_ESSENTIAL_raw)
agrega_esencialidad(G_LIT_Reguly,yeast_G_ESSENTIAL_raw)

In [6]:
def maximo_grado(red):
    cantidad_nodos = red.number_of_nodes()
    maximo = 0
    i=0
    while i<cantidad_nodos:
        if list(red.degree)[i][1]>maximo:
            maximo = list(red.degree)[i][1]
        i+=1
    return maximo

In [7]:
def distribucion_grado_esenciales(red):
    numero_nodos = red.number_of_nodes()
    l = len(red)
    m = maximo_grado(red)
    vector_nodos_esenciales= np.zeros(m+1)
    vector_nodos_no_esenciales= np.zeros(m+1)
    vector_grados= np.zeros(m+1)
    i = 0
    while i < numero_nodos:
        grado = list(red.degree)[i][1]
        vector_grados[grado] += 1
        
        nombre = list(red)[i]
        esencialidad_nodo = red.node[nombre]['esencialidad']
        
        if (esencialidad_nodo==1):
            vector_nodos_esenciales[grado] += 1
        else:
            vector_nodos_no_esenciales[grado] += 1
        
        i+=1
    return (vector_grados,vector_nodos_esenciales,vector_nodos_no_esenciales) 

In [8]:
distribucion_APMS = distribucion_grado_esenciales(G_APMS)

In [9]:
def lista_ordenada(red):
    #ordeno la red de mayor a menor grado
    copia_red=red.copy()
    lista_ordenada = sorted(copia_red.degree, key=lambda x: x[1], reverse=True)
    numero_nodos = red.number_of_nodes()
    m = maximo_grado(red)
    vector_indice_cambio_grado = np.zeros(m+1)
    grado_anterior = 0
    i = 0
    while i < numero_nodos:
        grado = list(lista_ordenada)[i][1]
        if grado != grado_anterior:      #me fijo en que indice cambia el grado
            vector_indice_cambio_grado[grado] = i 
            grado_anterior = grado
        i+=1    
    
    return(lista_ordenada,vector_indice_cambio_grado)

In [10]:
def analisis(red):
    copia_red=red.copy()
    numero_nodos = red.number_of_nodes()
    distribucion = distribucion_grado_esenciales(copia_red)
    ordenamiento = lista_ordenada(copia_red)
    red_ordenada = ordenamiento[0]
    vector_indice_cambio_grado = ordenamiento[1]
    eliminados = np.zeros(len(distribucion[2][:]))
    grado_minimo = 0
    grado_maximo = maximo_grado(red)

    i = 0 
    contador = 0
    vector_nombres = []
    
    while i < numero_nodos:
        nombre = list(copia_red)[i]
        esencialidad_nodo = copia_red.node[nombre]['esencialidad']
        if esencialidad_nodo == 1:
            grado = list(copia_red.degree)[i][1]
            nodos_no_esenciales_mismo_grado = int(distribucion[2][grado])

            if nodos_no_esenciales_mismo_grado > eliminados[grado]: #si hay por lo menos uno busco todos ellos directamente
                nodos_general_mismo_grado = int(distribucion[0][grado]) #nodos del mismo grado
                indice = int(vector_indice_cambio_grado[grado]) #indice a partir del cual el grado se mantiene
                c = 0
                vector = []
                while c < nodos_general_mismo_grado:
                    nuevo_nombre = red_ordenada[indice+c][0]
                    nuevo_grado = red_ordenada[indice+c][1]
                    nueva_esencialidad_nodo = copia_red.node[nuevo_nombre]['esencialidad']
                    if nueva_esencialidad_nodo == 0 and not nuevo_nombre in vector_nombres:
                        vector += [nuevo_nombre]
                    c+=1    
                nombre_nodo_aleatorio = random.choice(vector)  
                vector_nombres += [nombre_nodo_aleatorio]
                eliminados[grado] += 1
            else:
                
                nodo_superior = 0
                d = 0
                e = 1
                while d == 0:
                    grado_superior = grado + e
                    if grado_superior < grado_maximo:
                        nodos_no_esenciales_grado_superior = int(distribucion[2][grado_superior])
                        if vector_indice_cambio_grado[grado_superior]>0 and nodos_no_esenciales_grado_superior > eliminados[grado_superior]:
                            d = 1
                        else:
                            e += 1
                    else:
                        d = 1
                        nodo_superior = 1
                
                if nodo_superior == 0:
                    nodos_general_grado_superior = int(distribucion[0][grado_superior]) #nodos del mismo grado
                    indice = int(vector_indice_cambio_grado[grado_superior]) #indice a partir del cual el grado se mantiene
                    c = 0
                    vector = []

                    while c < nodos_general_grado_superior:
                        nuevo_nombre = red_ordenada[indice+c][0]
                        nuevo_grado = red_ordenada[indice+c][1]
                        nueva_esencialidad_nodo = copia_red.node[nuevo_nombre]['esencialidad']
                        if nueva_esencialidad_nodo == 0 and not nuevo_nombre in vector_nombres:
                            vector += [nuevo_nombre]
                        c+=1  
                    nombre_nodo_aleatorio = random.choice(vector)  
                    vector_nombres += [nombre_nodo_aleatorio]
                    eliminados[grado_superior] += 1
                else:
                    contador+=1
                    nodo_inferior = 0
                    d = 0
                    e = 1
                    while d == 0:
                        grado_inferior = grado - e
                        if grado_inferior > grado_minimo:
                            nodos_no_esenciales_grado_inferior = int(distribucion[2][grado_inferior])
                            if vector_indice_cambio_grado[grado_inferior]>0 and nodos_no_esenciales_grado_inferior > eliminados[grado_inferior]:
                                d = 1
                            else:
                                e += 1
                        else:
                            d = 1
                            nodo_inferior = 1

                    if nodo_inferior == 0:
                        nodos_general_grado_inferior = int(distribucion[0][grado_inferior]) #nodos del mismo grado
                        indice = int(vector_indice_cambio_grado[grado_inferior]) #indice a partir del cual el grado se mantiene
                        c = 0
                        vector = []

                        while c < nodos_general_grado_inferior:
                            nuevo_nombre = red_ordenada[indice+c][0]
                            nuevo_grado = red_ordenada[indice+c][1]
                            nueva_esencialidad_nodo = copia_red.node[nuevo_nombre]['esencialidad']
                            if nueva_esencialidad_nodo == 0 and not nuevo_nombre in vector_nombres:
                                vector += [nuevo_nombre]
                            c+=1 
                        nombre_nodo_aleatorio = random.choice(vector)  
                        vector_nombres += [nombre_nodo_aleatorio]
                        eliminados[grado_inferior] += 1
                
        i+=1
    return vector_nombres
        

In [11]:
def remove_non_essential(red):
    copia_red=red.copy()
    nodos_a_remover = analisis(red)
    l = len(nodos_a_remover)
    i = 0
    while i < l:
        nombre = nodos_a_remover[i]
        copia_red.remove_nodes_from([nombre])
        i+=1
    
    comp_gigante_removida = max(nx.connected_component_subgraphs(copia_red), key=len)
    nodos_comp_gigante_removida = comp_gigante_removida.number_of_nodes()
    return nodos_comp_gigante_removida

In [12]:
def iter_remove_no_esential(red):
    copia_red=red.copy()
    comp_gigante_original = max(nx.connected_component_subgraphs(copia_red), key=len)
    nodos_comp_gigante_original = float(comp_gigante_original.number_of_nodes())
    
    it = 10
    i = 0
    comp_gigante = []
    while i < it:
        comp_gigante += [remove_non_essential(copia_red)]
        i+=1  
    resultado = [x / nodos_comp_gigante_original for x in comp_gigante]
    medio = np.mean(resultado)
    desviacion = np.std(resultado)
    return(medio,desviacion)

In [13]:
resultado_remove_no_esential_APMS = iter_remove_no_esential(G_APMS)
resultado_remove_no_esential_Y2H = iter_remove_no_esential(G_Y2H)
resultado_remove_no_esential_LIT = iter_remove_no_esential(G_LIT)
resultado_remove_no_esential_LIT_Reguly = iter_remove_no_esential(G_LIT_Reguly)

In [14]:
def rem_esenciales(red):

    copia_red=red.copy()
    comp_gigante_original = max(nx.connected_component_subgraphs(copia_red), key=len)
    nodos_comp_gigante_original = float(comp_gigante_original.number_of_nodes())
    
    copia_red.remove_nodes_from(yeast_G_ESSENTIAL_raw[0][:])

    comp_gigante_removida = max(nx.connected_component_subgraphs(copia_red), key=len)
    nodos_comp_gigante_removida = comp_gigante_removida.number_of_nodes()

    return nodos_comp_gigante_removida/float(nodos_comp_gigante_original)


In [15]:
rem_essential_APMS = rem_esenciales(G_APMS)
rem_essential_Y2H = rem_esenciales(G_Y2H)
rem_essential_LIT = rem_esenciales(G_LIT)
rem_essential_LIT_Reguly = rem_esenciales(G_LIT_Reguly)

In [16]:
data = pd.DataFrame({" ":["AP/MS",'Y2H','LIT','LIT-Reguly'],
                     "Essential":[rem_essential_APMS,rem_essential_Y2H,rem_essential_LIT,rem_essential_LIT_Reguly],
                     "Random nonessential":[resultado_remove_no_esential_APMS[0],resultado_remove_no_esential_Y2H[0],resultado_remove_no_esential_LIT[0],resultado_remove_no_esential_LIT_Reguly[0]],
                     })
data

,,Essential,Random nonessential
0,AP/MS,0.322709,0.355179
1,Y2H,0.619308,0.618640
2,LIT,0.279472,0.417312
3,LIT-Reguly,0.576613,0.580583
